In [58]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import mysql.connector
import json

# Establish a connection to the MySQL database
connection = mysql.connector.connect(
    host='127.0.0.1',
    port=13306,
    user='root',
    password='root',
    database='pyml'
)

# Read the table data using pandas
query = """SELECT vi.hash_id, vi.contents as question_content, JSON_UNQUOTE(JSON_EXTRACT(tags, '$[*].slug')) AS slug FROM viblo_interview vi"""
df_ques = pd.read_sql(query, connection)

query_ans = """SELECT va.hash_id, va.contents, va.question_id FROM viblo_answer va"""
df_answer = pd.read_sql(query_ans, connection)

# Close the database connection
connection.close()

/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_67008/3451300885.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ques = pd.read_sql(query, connection)
/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_67008/3451300885.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_answer = pd.read_sql(query_ans, connection)


In [59]:
print(df_ques[600:610])

         hash_id                                   question_content  \
600  aAY4qdPDVPw  Làm thế nào để tối ưu hóa tài nguyên trong Kub...   
601  EbNVQoBAJvR  Làm thế nào để triển khai một ứng dụng đa vùng...   
602  y3RL1YGPLao  Làm thế nào để xử lý tình huống khi một Pod tr...   
603  PAoJezRkL1j  Bạn có thể giải thích sự khác biệt giữa "Kuber...   
604  Rk74ag8DLeO  Bạn có thể giải thích khái niệm "Kubernetes Cr...   
605  PwlVmwPgJ5Z  Có thể sử dụng Kubernetes để triển khai các ứn...   
606  zXRJ80x2JGq  Làm thế nào để triển khai và quản lý các ứng d...   
607  E1XVOM2XJMz  Khi nào bạn cần thực hiện Performance Testing ...   
608  WR5JRMyD4Gv  Khi nào bạn cần thực hiện Load Testing cho hệ ...   
609  y37Ldgo0Jov      So sánh Load Testing với Performance Testing?   

                  slug  
600  b'["kubernetes"]'  
601  b'["kubernetes"]'  
602  b'["kubernetes"]'  
603  b'["kubernetes"]'  
604  b'["kubernetes"]'  
605  b'["kubernetes"]'  
606  b'["kubernetes"]'  
607     b'["testin

In [63]:
def filter_slug(x):
    if x is None:
        return ''
    t = str(x).replace("b'", "").replace("'", "")
    t1 = json.loads(t)

    return t1
df_ques['slug_filter'] = df_ques['slug'].apply(filter_slug)
print(df_ques['slug_filter'][0])

['javascript']


In [70]:

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=2500, min_df=7, max_df=0.8)
# X_train_tfidf = vectorizer.fit_transform(df_ques['contents']).toarray()
X = vectorizer.fit_transform(df_ques['question_content']).toarray()

Y = df_ques['slug_filter']
name_counts = df_ques['slug_filter'].value_counts()
print(name_counts)
print('y_train', Y)

[java]                                112
[devops]                              109
[backend-development]                  94
[soft-skills]                          84
[git]                                  69
                                     ... 
[sql, backend-development]              1
[devops, backend-development]           1
[clean-code]                            1
[design-pattern, programming]           1
[object-oriented-programming, php]      1
Name: slug_filter, Length: 130, dtype: int64
y_train 0           [javascript]
1       [cyber-security]
2       [cyber-security]
3       [cyber-security]
4       [cyber-security]
              ...       
1663            [golang]
1664            [golang]
1665            [golang]
1666            [golang]
1667            [golang]
Name: slug_filter, Length: 1668, dtype: object


In [84]:
print(df_ques.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1668 entries, 0 to 1667
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   hash_id           1668 non-null   object
 1   question_content  1668 non-null   object
 2   slug              1667 non-null   object
 3   slug_filter       1668 non-null   object
dtypes: object(4)
memory usage: 52.2+ KB
None


In [77]:
from sklearn.preprocessing import MultiLabelBinarizer

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
# Convert the input data to a binary array
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(y_train)
y_test = mlb.transform(y_test)

print('y_test', y_test[0])

y_test [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [85]:
t = y_train.ravel()
print('t', t.shape, y_train.shape, X_train.shape)

t (117392,) (1334, 88) (1334, 450)


In [86]:
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier

svc = SVC(kernel='linear')
multi_output_svc = MultiOutputClassifier(svc)

# Fit the classifier on the training data
multi_output_svc.fit(X_train, y_train)

MultiOutputClassifier(estimator=SVC(kernel='linear'))

In [89]:
from sklearn.metrics import accuracy_score

y_test_pred = multi_output_svc.predict(X_test)

print(y_test_pred[:2])
# Evaluate accuracy
accuracy = accuracy_score(y_test, y_test_pred)
print("Accuracy:", accuracy)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Accuracy: 0.38323353293413176
